# L1/L2 regularization
需要restart，再重新运行，不然placeholder的赋值会有问题。。。

In [1]:
import tensorflow as tf
from numpy.random import RandomState

def get_weight(shape, lamda):
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
#    tf.add_to_collection(
#    "losses", tf.contrib.layers.l2_regularizer(lamda)(var))
    return var

x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")

batch_size = 8

layer_dimension = [2, 10, 10, 10, 1]

n_layers = len(layer_dimension)

cur_layer = x

in_dimension = layer_dimension[0]

for i in range(1, n_layers):
    out_dimension = layer_dimension[i]
    weight = get_weight([in_dimension, out_dimension], 0.0001)
    bias = tf.Variable(tf.constant(0.1, shape=[out_dimension]))
    cur_layer = tf.nn.relu(tf.matmul(cur_layer, weight) + bias)
    in_dimension = layer_dimension[i]
    
y = cur_layer
    
mse_loss = tf.reduce_mean(tf.square(y_ - y))

cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)) 
                               + (1 - y_) * tf.log(tf.clip_by_value(1-y, 1e-10, 1.0)))

#tf.add_to_collection("losses", mse_loss)

tf.add_to_collection("losses", cross_entropy)

loss = tf.add_n(tf.get_collection("losses"))


dataset_size = 128

rdm = RandomState(1)
X = rdm.rand(dataset_size, 2)

Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]

#Y = [[(x1 + x2) * 1.0] for (x1, x2) in X]


train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)

    
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        #print(X[start:end].shape)
        #print(Y[start:end])
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(
            loss, feed_dict={x: X, y_: Y})
            print("After %d training steps, cross entropy on all data is %g" % (i, total_cross_entropy))


After 0 training steps, cross entropy on all data is 10.8458
After 1000 training steps, cross entropy on all data is 6.21182
After 2000 training steps, cross entropy on all data is 5.14197
After 3000 training steps, cross entropy on all data is 2.63666
After 4000 training steps, cross entropy on all data is 0.986491
